Created by [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [RI Law Revision](http://webserver.rilin.state.ri.us/Lawrevision/exeap2019.htm).
<hr>

# Rhode Island Executive Appointments

Rhode Island has a list of executive appointments since 2000. This data includes appointments, <br>
appointee's, date and expiration of appointment. Early data also includes the city the appointee <br>
is from and later data includes the legislative authority which allows the appointment.


To use this you will have to download [selenium](https://selenium-python.readthedocs.io/), [pandas](https://pandas.pydata.org/pandas-docs/stable/) and [chromedriver](https://sites.google.com/a/chromium.org/chromedriver/).

## Collecting Data

The code below uses selenium to webscrape all the tables for each year, <br>
converts them to a pandas dataframe, combines them all into 1 and saves it in `/data/raw`.

In [1]:
import pandas as pd
from selenium import webdriver

In [2]:
browser = webdriver.Chrome('C:/Users/Alex/bin/chromedriver.exe')
url = 'http://webserver.rilin.state.ri.us/Lawrevision/exeap'

# get start and end year
start = 2000
end = 2019

frames = []
for year in range(start, end + 1):
    query_url = url + str(year) + '.htm'
    browser.get(query_url)
    
    # get the table and load it as a pandas dataframe
    table = browser.find_elements_by_tag_name('table')[2]
    table = table.get_attribute('outerHTML')
    df = pd.read_html(table)[0]
    
    # some years have a blank table row (removes them)
    df = df.dropna(axis=1, how='all')
    
    headers = ['appointment', 'appointee', 'date', 'expires']
    
    if len(list(df)) > 4:
        headers.insert(2, 'authority')
        df.columns = headers
        
    else:
        df.columns = headers
        
    # drop headers row as some are missing it 
    df = df[~(df['appointee'] == 'APPOINTEE')]
    
    # add a year column
    df['year'] = str(year)
    
    # add dataframe to frames list
    frames.append(df)
    
# combine all frames into 1 dataset
frames = pd.concat(frames, sort=True)

# saving file
frames = frames.to_csv('./data/raw/executive_appointments_raw.csv', index=False)

## Cleaning Data

The code below uses pandas to clean the raw data and produce a clean one by doing the following:

- adds seperate column for lifetime, governor's pleasure, ect. named `extra_expires`.
- splits appointee on the word "of" which is followed by location creating a `appointee_location`.
- adds governor and party info for each year named `governor` & `party`.

In [3]:
df = pd.read_csv('./data/raw/executive_appointments_raw.csv')

# remove double spaces
df['appointee'] = df['appointee'].str.replace('  ', ' ')
df['appointment'] = df['appointment'].str.replace('  ', ' ')

# add extra info from expires
df['expires'] = df['expires'].str.lower()

df.loc[(df['expires'].str.contains('life', na=False)), 'extra_expires'] = 'lifetime'
df.loc[(df['expires'].str.contains('governor', na=False)), 'extra_expires'] = 'governor’s pleasure'
df.loc[(df['expires'].str.contains('senate', na=False)), 'extra_expires'] = 'senate, advice & consent'
df.loc[(df['expires'].str.contains('town', na=False)), 'extra_expires'] = 'town council representative'
df['extra_expires'] = df['extra_expires'].fillna('other')

# split on "of" which shows location of appointee (missing in later years)
df.loc[(df['appointee'].str.contains('of', na=False)), 'appointee_location'] = df["appointee"].str.split(" of ").str[-1]
df.loc[(df['appointee'].str.contains('of', na=False)), 'appointee'] = df["appointee"].str.split(" of ").str[0]
df['appointee_location'] = df['appointee_location'].fillna('n/a')

# adding governor and party info
df.loc[(df['year'] >= 2000) & (df['year'] <= 2002), 'governor'] = 'Lincoln Almond'
df.loc[(df['year'] >= 2000) & (df['year'] <= 2002), 'party'] = 'Republican'

df.loc[(df['year'] >= 2003) & (df['year'] <= 2010), 'governor'] = 'Donald Carcieri'
df.loc[(df['year'] >= 2003) & (df['year'] <= 2010), 'party'] = 'Republican'

df.loc[(df['year'] >= 2011) & (df['year'] <= 2012), 'governor'] = 'Lincoln Chafee'
df.loc[(df['year'] >= 2011) & (df['year'] <= 2012), 'party'] = 'Independent'

df.loc[(df['year'] >= 2013) & (df['year'] <= 2014), 'governor'] = 'Lincoln Chafee'
df.loc[(df['year'] >= 2013) & (df['year'] <= 2014), 'party'] = 'Democrat'

df.loc[(df['year'] >= 2015) & (df['year'] <= 2019), 'governor'] = 'Gina Raimondo'
df.loc[(df['year'] >= 2015) & (df['year'] <= 2019), 'party'] = 'Democrat'

# save output file
df.to_csv('./data/clean/executive_appointments_clean.csv', index=False)